# Analysis of Twitter and NIST CVE references

In [1]:
import requests
import time
import re
import psycopg2
import os

import plotly.plotly as py
import plotly.graph_objs as go

import cufflinks as cf
import pandas as pd

from scipy import stats

PLOTLY_UN    = os.environ.get("PLOTLY_UN")
PLOTLY_TOKEN = os.environ.get("PLOTLY_TOKEN")
#plotly.tools.set_credentials_file(username=PLOTLY_UN, api_key=PLOTLY_TOKEN)

POSTGRES_DB_NAME = os.environ.get("API_TOKEN")
POSTGRES_DB_UN   = os.environ.get("POSTGRES_DB_UN")
POSTGRES_DB_PW   = os.environ.get("POSTGRES_DB_PW")
POSTGRES_DB_HOST = os.environ.get("POSTGRES_DB_HOST")
%load_ext sql
%config echo=False
%sql postgresql://$POSTGRES_DB_UN:$POSTGRES_DB_PW@$POSTGRES_DB_HOST/$POSTGRES_DB_NAME
connection = psycopg2.connect(dbname=POSTGRES_DB_NAME, user=POSTGRES_DB_UN, password=POSTGRES_DB_PW);
cursor     = connection.cursor()
#connect_to_db = 'postgresql+psycopg2://' + \
                #creds['user_name'] + ':' + creds['password'] + '@' + \
                #creds['host_name'] + ':' + creds['port_num'] + '/' + creds['db_name'];

MONGO_DB_NAME = os.environ.get("MONGO_DB_NAME")
MONGO_DB_UN   = os.environ.get("MONGO_DB_UN")
MONGO_DB_PW   = os.environ.get("MONGO_DB_PW")
MONGO_DB_HOST = os.environ.get("MONGO_DB_HOST")

#%sql SELECT relname, reltuples FROM pg_class WHERE relname IN (SELECT table_name FROM information_schema.tables WHERE table_schema = 'public') ORDER BY reltuples DESC

## Finding proven and time-tested sources seems unlikely because of URL shorteners

In [2]:
query = """
SELECT *
FROM (
    SELECT ted.domain,
           extract(year from t.timestamp) AS t_year,
           SUM(t.retweet_count) AS rt_cnt,
           SUM(t.favorite_count) AS fav_cnt,
           COUNT(ted.domain) AS cnt 
    FROM view_cve_referring_tweets_extracted_domains ted 
    JOIN cve_referring_tweets t 
    ON t.id = ted.tweet_id 
    WHERE character_length(ted.domain::varchar) > 6
    AND ted.domain <> 'web.nvd.nist.gov'
    AND ted.domain <> 'nvd.nist.gov'
    AND ted.domain <> 'dlvr.it'
    AND ted.domain <> 'twitter.com'
    GROUP BY ted.domain, t_year 
    ORDER BY t_year DESC, cnt ASC 
) a 
WHERE cnt > 150"""

df = pd.read_sql_query(query, connection)
df = df.sort_values(by=['domain'], ascending = False)

lyt = go.Layout(
    title='Twitter reference sources by year and frequency',
    autosize=False,
    height=1250,
    margin=go.Margin(
      l=175
    )
)

data = [
    {
        'x': df.t_year,
        'y': df.domain,
        'mode': 'markers',
        'marker': {
            'color': df.cnt,
            'size': 10,
            'showscale': True
        }
    }
]

fig = go.Figure(data = data, layout = lyt)
py.iplot(fig, filename='twitter-source-cwe-popularity-bubble-chart')

## Picking a good source for specific CWE information is hard due to URL shorteners, too

In [3]:
query = """
SELECT *
FROM (
    SELECT ted.domain,
           cvecwe.cweid,
           COUNT(ted.domain) AS cnt 
    FROM view_cve_referring_tweets_extracted_domains ted 
    JOIN view_cve_referring_tweets_extracted_cves tec ON ted.tweet_id = tec.tweet_id 
    JOIN cve_cwe_classification cvecwe ON tec.cve = cvecwe.cveid
    
    WHERE character_length(ted.domain::varchar) > 6
    AND ted.domain <> 'web.nvd.nist.gov'
    AND ted.domain <> 'nvd.nist.gov'
    AND ted.domain <> 'dlvr.it'
    AND ted.domain <> 'twitter.com'
    
    GROUP BY ted.domain, cvecwe.cweid
    ORDER BY cnt ASC 
) a 
WHERE cnt > 150"""

df = pd.read_sql_query(query, connection)
df = df.sort_values(by=['domain'], ascending = False)

lyt = go.Layout(
    title='Twitter reference sources by CWE and frequency',
    autosize=False,
    height=750,
    margin=go.Margin(
      l=175
    )
)

data = [
    {
        'x': df.cweid,
        'y': df.domain,
        'mode': 'markers',
        'marker': {
            'color': df.cnt,
            'size': 10,
            'showscale': True
        }
    }
]

fig = go.Figure(data = data, layout = lyt)
py.iplot(fig, filename='twitter-source-cwe-bubble-chart')

## Official NIST references are better suited to pick good CWE specific sources

In [4]:
query = """
SELECT *, nrefs/nrefstotal::float AS refshare
FROM (
    SELECT DISTINCT cr.domain,
           cvecwe.cweid,
           COUNT(cr.domain) OVER (PARTITION BY cvecwe.cweid) AS nrefstotal,
           COUNT(cr.domain) OVER (PARTITION BY cr.domain, cvecwe.cweid) AS nrefs
    FROM view_cvereference_extracted_domains cr 
    JOIN cve_cwe_classification cvecwe 
    ON cr.cveid = cvecwe.cveid 
    ORDER BY nrefs DESC
) a
WHERE nrefs/nrefstotal::float > 0.25"""

df = pd.read_sql_query(query, connection)
df = df.sort_values(by=['domain'], ascending = False)

lyt = go.Layout(
    title='NIST reference sources by CWE and share of total CWE references',
    autosize=False,
    height=600,
    margin=go.Margin(
      l=175
    )
)

data = [
    {
        'x': df.cweid,
        'y': df.domain,
        'mode': 'markers',
        'marker': {
            'color': df.nrefs,
            'size': 10,
            'showscale': True
        }
    }
]

fig = go.Figure(data = data, layout = lyt)
py.iplot(fig, filename='nist-reference-cwe-popularity-bubble-chart')

## A small number of Github users is also active on Twitter under the same handle

In [5]:
query = """
SELECT
    DISTINCT t.username,
    extract(year from t.timestamp) AS t_year,
    COUNT(t.id) OVER (PARTITION BY t.username, extract(year from t.timestamp)) AS t_user_count,
    COUNT(t.id) OVER (PARTITION BY extract(year from t.timestamp)) AS t_year_count
FROM cve_referring_tweets t
JOIN view_commit_data_search_for_cve vc ON vc.name = t.username
ORDER BY t.username, t_year"""

df = pd.read_sql_query(query, connection)

lyt = go.Layout(
    title='Same Github & Twitter handles over time and share-of-year-volume',
    autosize=False,
    height=1000,
    margin=go.Margin(
      l=175
    )
)

data = [
    {
        'x': df.t_year,
        'y': df.username,
        'mode': 'markers',
        'marker': {
            'color': df.t_user_count / df.t_year_count,
            'size': 10,
            'showscale': True
        }
    }
]

fig = go.Figure(data = data, layout = lyt)
py.iplot(fig, filename='same-userhandles-time-volume-bubble-chart')

## And apparently, some are knowledgeable on certain types of bugs

In [6]:
query = """
SELECT
    DISTINCT t.username,
    ccc.cweid,
    COUNT(t.id) OVER (PARTITION BY t.username, ccc.cweid) AS t_cwe_count,
    COUNT(t.id) OVER (PARTITION BY ccc.cweid) AS t_count
FROM cve_referring_tweets t
JOIN view_commit_data_search_for_cve vc ON vc.name = t.username
JOIN view_cve_referring_tweets_extracted_cves ec ON t.id = ec.tweet_id
JOIN cve_cwe_classification ccc ON ec.cve = ccc.cveid"""

df = pd.read_sql_query(query, connection)

lyt = go.Layout(
    title='Same Github & Twitter handles over time and share-of-cwe-volume',
    autosize=False,
    height=1700,
    margin=go.Margin(
      l=175
    )
)

data = [
    {
        'x': df.cweid,
        'y': df.username,
        'mode': 'markers',
        'marker': {
            'color': df.t_cwe_count / df.t_count,
            'size': 10,
            'showscale': True
        }
    }
]

fig = go.Figure(data = data, layout = lyt)
py.iplot(fig, filename='same-userhandles-cwe-volume-bubble-chart')

## Which products are affected a lot by certain CPE? 

In [7]:
query = """
SELECT *
FROM (
    SELECT DISTINCT
        cpv.name,
        ccc.cweid,
        COUNT(cpv.cveid) OVER (PARTITION BY cpv.name) AS total_cvecount,
        COUNT(cpv.cveid) OVER (PARTITION BY cpv.name, ccc.cweid) AS cwe_cvecount
    FROM cve_per_product_version cpv
    JOIN cve_cwe_classification ccc ON ccc.cveid = cpv.cveid
) a
ORDER BY total_cvecount DESC, cwe_cvecount DESC
LIMIT 200"""

df = pd.read_sql_query(query, connection)
df = df.reset_index().groupby(['name', 'cweid'])['cwe_cvecount'].aggregate('first').unstack()

data = [go.Bar(x=df.index, y=df[k], name='CWE {0}'.format(k)) for k in df.keys()]

layout = go.Layout(
    barmode='stack',
    title='CWE distribution per product',
    height=750
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='cwe-distribution-per-product')